# Data preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

In [6]:
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [7]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [8]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [9]:
np.set_printoptions(suppress=True, formatter={'float': '{: 0.2f}'.format})
X

array([[ 1.00,  0.00,  0.00, ...,  1.00,  1.00,  101348.88],
       [ 0.00,  0.00,  1.00, ...,  0.00,  1.00,  112542.58],
       [ 1.00,  0.00,  0.00, ...,  1.00,  0.00,  113931.57],
       ...,
       [ 1.00,  0.00,  0.00, ...,  0.00,  1.00,  42085.58],
       [ 0.00,  1.00,  0.00, ...,  1.00,  0.00,  92888.52],
       [ 1.00,  0.00,  0.00, ...,  1.00,  0.00,  38190.78]])

In [10]:
X = X[:, 1:]

In [11]:
X

array([[ 0.00,  0.00,  619.00, ...,  1.00,  1.00,  101348.88],
       [ 0.00,  1.00,  608.00, ...,  0.00,  1.00,  112542.58],
       [ 0.00,  0.00,  502.00, ...,  1.00,  0.00,  113931.57],
       ...,
       [ 0.00,  0.00,  709.00, ...,  0.00,  1.00,  42085.58],
       [ 1.00,  0.00,  772.00, ...,  1.00,  0.00,  92888.52],
       [ 0.00,  0.00,  792.00, ...,  1.00,  0.00,  38190.78]])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# ANN

In [14]:
import keras

Using TensorFlow backend.


In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [30]:
# Initialising the ANN
classifier = Sequential()

In [31]:
X_train.shape

(8000, 11)

In [ ]:
# Layers, first with dropout
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_shape=(11,)))
classifier.add(Dropout(rate=0.1))

classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate=0.1))

classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

In [ ]:
#Compile
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Fit
classifier.fit(X_train, y_train, batch_size=10, epochs=20)

In [ ]:
# Predicting

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
np.set_printoptions(suppress=True, threshold=np.inf, formatter={'float': '{: 0.2f}'.format})

y_pred[0:10]

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
# Accuracy
(1550 + 175)/2000

In [ ]:
# Evaluating ANN

In [32]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV

In [35]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_shape=(11,)))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [36]:
classifier = KerasClassifier(build_fn=build_classifier)

In [34]:
parameters = {'batch_size':[25,32],
              'nb_epoch':[10,20],
             'optimizer':['adam','rmsprop',]}

In [37]:
grid_search = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                          scoring='accuracy',
                          cv=10)

In [39]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/1
7200/7200 [==============================] - 3s 396us/step - loss: 0.5526 - acc: 0.7961
Epoch 1/1
7200/7200 [==============================] - 3s 383us/step - loss: 0.5759 - acc: 0.7950
Epoch 1/1
7200/7200 [==============================] - 3s 446us/step - loss: 0.5650 - acc: 0.7944
Epoch 1/1
7200/7200 [==============================] - 4s 497us/step - loss: 0.5417 - acc: 0.7962
Epoch 1/1
7200/7200 [==============================] - 3s 373us/step - loss: 0.5540 - acc: 0.7935
Epoch 1/1
7200/7200 [==============================] - 3s 435us/step - loss: 0.5733 - acc: 0.7938
Epoch 1/1
7200/7200 [==============================] - 3s 433us/step - loss: 0.5517 - acc: 0.7969
Epoch 1/1
7200/7200 [==============================] - 3s 466us/step - loss: 0.5549 - acc: 0.7958
Epoch 1/1
7200/7200 [==============================] - 3s 429us/step - loss: 0.5649 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 3s 483us/step - loss: 0.5568 - acc: 0.7950
Epoch 1/1
7200/7200 

KeyboardInterrupt: 

In [ ]:
best_parameters

In [20]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=1)
accuracies

array([ 0.79,  0.79,  0.80,  0.78,  0.82,  0.81,  0.79,  0.79,  0.80,
        0.80])

In [21]:
mean = np.mean(accuracies)

In [24]:
std = accuracies.std()

In [25]:
mean

0.7960000026226044

In [26]:
std

0.010105689834872308

In [27]:
#Dropout regularization